In [ ]:
# %load_ext autoreload
# %autoreload 2
import kalman_reconstruction
from kalman_reconstruction.kalman import (
    Kalman_SEM,
    Kalman_filter,
    Kalman_smoother,
    )
from kalman_reconstruction.statistics import (
    gaussian_weights
    )

from scipy.integrate import odeint
from sklearn.linear_model import LinearRegression
import numpy as np
import scipy as sp
import random
import importlib
from tqdm import tqdm

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 5)
# Set default matplotlib style
MEDIUM_SIZE = 12
BIGGER_SIZE = 15
plt.style.use('seaborn-v0_8-whitegrid')
# plt.style.use('dark_background')

plt.rcParams['figure.figsize'] = (10.0, 6.0)
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('legend', loc='upper right')
# use colorblind save colors https://davidmathlogic.com/colorblind
colors = [
    '#CC6677',
    '#44AA99',
    '#DDCC77',
    '#6E9CB3',
    '#AA4499',
    '#A494F5',
]
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=colors) 


In [ ]:

def Kalman_SEM_timedependent(x, y, H, R, nb_iter_SEM, alpha = 0.2):  # , x_t, t):
    """Apply the stochastic expectation-maximization algorithm."""

    # fix the seed
    np.random.seed(11)

    # copy x
    x_out = x.copy()

    # shapes
    dim0_length = np.shape(x_out)[0]
    n = np.shape(x_out)[1]

    # tab to store the np.log-likelihood
    tab_loglik = []
    M = np.empty((dim0_length, n, n))

    # loop on the SEM iterations
    for i in tqdm(np.arange(0, nb_iter_SEM)):
        for x_idx in tqdm(range(0, dim0_length)) :
            # get the sample weights 
            sample_weight = gaussian_weights(x_out[:-1,], x_out[x_idx,], axis = 1)
            # Kalman parameters
            reg = LinearRegression(fit_intercept=False).fit(x_out[:-1,], x_out[1:,], sample_weight=sample_weight)
            M[x_idx,] = reg.coef_
            Q = np.cov((x_out[1:,] - reg.predict(x_out[:-1,])).T)
            # R   = np.cov(y.T - H @ x.T)

        # Kalman initialization
        if i == 0:
            x0 = np.empty(n)
            P0 = np.eye(n)
        else:
            x0 = x_s[0, :]
            P0 = P_s[
                0,
                :,
                :,
            ]

        # # apply the Kalman smoother
        # x_f, P_f, x_a, P_a, x_s, P_s, loglik, P_s_lag = Kalman_smoother(
        #     y, x0, P0, M, Q, H, R
        # )

        # # store the np.log-likelihod
        # tab_loglik = np.append(tab_loglik, sum(loglik))

        # # simulate the new x
        # for k in range(len(x_s)):
        #     x_out[k, :] = np.random.multivariate_normal(x_s[k, :], P_s[k, :, :])

    return x_s, P_s, M, tab_loglik, x_out, x_f, Q


In [ ]:
# generate a and b
np.random.seed(0) # setting seed for reproducability
amount = 100 # number of sample
components = 2
length = 2*np.pi

coords_x = np.linspace(0, length, amount)
coords_y = np.arange(components)

x_test = np.zeros((amount, components)) + coords_x[:,np.newaxis]
y_test = np.sin(x_test) + np.random.normal(0, 0.25, size=np.shape(x_test))

x_0_test = (1 + coords_y) * 2
weights = gaussian_weights(x_test, x_0_test, axis = 1, alpha = 0.3)
fig, axs = plt.subplots(nrows =3)
axs = axs.flatten()
axs[0].pcolor(coords_x, coords_y, y_test.T, cmap = 'RdBu_r', vmin = -1, vmax = 1)
axs[1].pcolor(coords_x, coords_y, weights.T, cmap = "Reds")
axs[2].pcolor(coords_x, coords_y, (y_test * weights).T, cmap = "RdBu_r", vmin = -0.1, vmax = 0.1)
plt.plot(x_0_test, coords_y, 'X', color='w', label = 'Center of Gausian')
for ax in axs:
    ax.set_ylabel('component index')
    ax.set_xlabel('time')

fig.suptitle("2D linear regression problem\nusing 2D weights")
fig.tight_layout()
fig.savefig('2D-linear-regression-problem.png')

In [ ]:
variance_obs_comp = 0.00001
nb_iter_SEM = 2
# shapes
n = np.shape(x_test)[1]
p = np.shape(y_test)[1]

# kalman parameters
H_test = np.eye(n)
R_test = variance_obs_comp*np.eye(p)

# stochasti

Kalman_SEM_timedependent(x_test, y_test, H_test, R_test, nb_iter_SEM, alpha = 0.2)

In [ ]:
x_test[:-1,].shape

### Important parameters

In [ ]:
# index of the unobserved component
i_unobs_comp = 0

# integration time step
dt = 0.01

# variance of the random white noise of z
variance_unobs_comp = 5

# variance of the observation error used in Kalman
variance_obs_comp = 0.0001

# number of Lorenz-63 times
nb_loop = 2

# number of SEM iterations
nb_iter_SEM = 30

### Generate simulated data

In [ ]:
# Lorenz-63 dynamical model
def Lorenz_63(x, dx, sigma, rho, beta):
    dx = np.empty((3))
    dx[0] = sigma*(x[1]-x[0])
    dx[1] = x[0]*(rho-x[2])-x[1]
    dx[2] = x[0]*x[1] - beta*x[2]
    return dx

# Lorenz-63 parameters
x0 = np.array([8, 0, 30]) # initial condition
sigma = 10; rho = 28; beta = 8/3 # physical parameters

# time and truth
t   = np.arange(0.01, nb_loop, dt)
x_t = odeint(Lorenz_63, x0, np.arange(0.01, nb_loop, dt), args=(sigma, rho, beta))

# observations
y = x_t.copy()
# remove unobserved component
y = np.delete(y, i_unobs_comp, axis=1) # add noise here?

In [ ]:
x_t.shape

## Time independent reconstruction

### V0: $x = [x_2, x_3]$

In [ ]:
# state
x = np.c_[y[:,0], y[:,1]]

# shapes
n = np.shape(x)[1]
p = np.shape(y)[1]

# colors and labels of the components
tab_labels = ['$x_2$', '$x_3$']

# plot the components
plt.plot(t, x)
plt.xlabel('System loops', size=20)
plt.ylabel('Components of the system', size=20)
plt.legend(tab_labels, loc=1, fontsize='xx-large')
plt.title('Observed components', size=30)

In [ ]:
# kalman parameters
H = np.eye(n)
R = variance_obs_comp*np.eye(p)

# stochastic EM
x_s_V0, P_s_V0, M_V0, loglik_V0, x, x_f_V0, Q_V0 = Kalman_SEM(x, y, H, R, nb_iter_SEM)

### V1: $x = [x_2, x_3, z_1]$

In [ ]:
# state
z = np.random.normal(loc=x_t[:,i_unobs_comp]*0, scale=variance_unobs_comp, size=np.shape(y)[0])
x = np.c_[y[:,0], y[:,1], z]

# shapes
n = np.shape(x)[1]
p = np.shape(y)[1]

In [ ]:
##################
### FIGURE 1-a ###
##################

tab_labels = ['$y_2 \ (2^{nd} \ \mathrm{Lorenz \ component})$', '$y_3 \ (3^{rd} \ \mathrm{Lorenz \ component})$', '$z_1 = Random(\mathcal{N}(0,\sigma^2))$']
for i in [2,0,1]:
    plt.plot(t, x[:,i], color=colors[i], label=tab_labels[i])
plt.xlabel('Time')
plt.legend(loc=1)
plt.ylim([-25,45])
plt.xlim([t[0],t[-1]])

In [ ]:
# kalman parameters
H = np.delete(np.eye(n), 2, axis=0)
R = variance_obs_comp*np.eye(p)

# stochastic EM
x_s_V1, P_s_V1, M_V1, loglik_V1, x, x_f_V1, Q_V1 = Kalman_SEM(x, y, H, R, nb_iter_SEM)

In [ ]:
##################
### FIGURE 1-b ###
##################

tab_labels = ['$x^s_2$', '$x^s_3$', '$x^s_1 \pm 1.96 \sqrt{P^s_1}$']
for i in [2,0,1]:
    plt.plot(t, x_s_V1[:,i], color=colors[i], label=tab_labels[i])
    plt.fill_between(t, x_s_V1[:,i]-1.96*np.sqrt(P_s_V1[:,i,i]), x_s_V1[:,i]+1.96*np.sqrt(P_s_V1[:,i,i]),
                 facecolor=colors[i], alpha=0.50)   
plt.xlabel('Time')
plt.legend(loc=1)
plt.ylim([-25,45])
plt.xlim([t[0],t[-1]])

## Time dependent Model

### test Gaussian weights function